In [1]:
%matplotlib widget
import ipywidgets as widgets

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.cm as cm
from scipy.interpolate import interp1d

In [3]:
# contains helper functions to do actual optimizaiton analysis
import optimization_helper_functions as OHF

In [4]:
cwd = os.getcwd()

In [5]:
diameter = 1.27 #cm
area = np.pi*(diameter/2)**2
Fconst=96485

molar_mass_LiV3O8 = 287.7607
density_LiV3O8 = 3.15              
density_carbon = 2.266
max_mAhg = 362

ed_col = 'Electrode Volumetric Energy Density (Wh/cm3)'
pd_col = 'Electrode Volumetric Power Density (W/cm3)'
aed_col = 'Electrode Areal Energy Density (Wh/cm2)'
apd_col = 'Electrode Areal Power Density (W/cm2)'
ced_col = 'Cell Volumetric Energy Density (Wh/cm3)'
cpd_col = 'Cell Volumetric Power Density (W/cm3)'
cap_col = 'Capacity (mAh/g)'

In [6]:
# def add_anode(sims,L_sep=20,L_cc_a=15,L_cc_c=15,Q_spec_a=372,rho_a=2.2,eps_a=0.35,Q_a_Q_c_ratio=1):
def add_anode(sims,L_sep=20,L_cc_a=15,L_cc_c=15,Q_spec_a=3860,rho_a=0.534,eps_a=0.0,Q_a_Q_c_ratio=1):

    LBOC = L_sep+L_cc_a+L_cc_c
    
    sims['L_anode (um)'] = 10000*max_mAhg*sims['AM Loading (g)']/(Q_a_Q_c_ratio*Q_spec_a*area*(1-eps_a)*rho_a)
    sims['L_cell (um)'] = sims['L_anode (um)'] + LBOC + sims['L (um)']
    
    sims[ced_col] = sims[aed_col]/((sims['L (um)']+LBOC+sims['L_anode (um)'])/10000)
    sims[cpd_col] = sims[apd_col]/((sims['L (um)']+LBOC+sims['L_anode (um)'])/10000)
    
    return sims

In [7]:
# read in simulations table
all_sims_that_ran = pd.read_csv('simulations_table.txt',sep=',')

# add anode
all_sims_that_ran = add_anode(all_sims_that_ran)

The details of this analysis can be found in the paper [Design Principles to Govern Electrode Fabrication for the Lithium Trivanadate Cathode](https://iopscience.iop.org/article/10.1149/1945-7111/ab91c8). 

# Optimal Design

Choose the **optimization metric** (i.e. Cell Energy Density) you wish to optimize. 
The **sensitivity** will be displayed as a shaded region that will show the range of design parameter values for which the designated fraction of the maximum achievable **optimization metric** is achieved when the other design parameters are held at their optimal values.
After the desired **optimization metric** and **sensitivity** are chosen, click the **Run Interact** button to update the plot. 

In [9]:
# plt.close(figi)
figi,axi = plt.subplots(1,3,figsize=(9,2.5),constrained_layout=True)
@widgets.interact_manual(optimization_metric=[ced_col,aed_col,ed_col,cpd_col,apd_col,pd_col],sensitivity=(0.5,1,0.05))
def plot_optimization(optimization_metric,sensitivity):
    for ax in axi:
        ax.clear()
        ax.grid(True)
    
#     # load all data
#     drop_cols = ['Bind Loading (g)', 'Cathode Volume (cm3)','Vol Frac Bind','mAh_exp','tstep (s)',
#              'Current (A)','Tortuosity (Bruggeman)']
#     crates = [0.1,0.2,0.33,0.5,0.75,1]
#     all_crate_sims_that_ran = []
#     for cr in crates:

#         colname_filename = cwd+'/vary_crate/'+str(cr)+'_C/column_names.txt'

#         filename = cwd+'/vary_crate/'+str(cr)+'_C/Simulation_Parameters_processed.txt'
#         all_sims, sims_that_ran, sims_that_crashed = OHF.read_in_sims_table(filename,colname_filename,drop_cols,print_cols=False)

#         sims_that_ran[apd_col] = sims_that_ran[pd_col]*sims_that_ran['L (um)']/10000
#         all_crate_sims_that_ran.append(sims_that_ran)

#     all_sims_that_ran = pd.concat(all_crate_sims_that_ran)
#     all_sims_that_ran = all_sims_that_ran.drop(['fail_id','Time','Voltage','Integrated Voltage','Ran','Utilization_cr'],axis=1)
#     all_sims_that_ran = add_anode(all_sims_that_ran)
        
    all_sims_that_ran = pd.read_csv('simulations_table.txt',sep=',')
    all_sims_that_ran = add_anode(all_sims_that_ran)
    design_parameter = optimization_metric
    best_cols = ['Porosity','Vol Frac Cond','L (um)']
    output_labels = [r'Cell $E_V$ ($\frac{Wh}{L}$)',r'$E_A$ ($\frac{Wh}{m^2}$)']

    if design_parameter != cap_col:
        all_sims_that_ran,new_design_parameter = OHF.change_units(all_sims_that_ran,design_parameter)
    else:
        new_design_parameter = design_parameter
    best_df = OHF.design_guide_plots(all_sims_that_ran,best_cols,[axi[0],axi[1],axi[2]],optimize_output=new_design_parameter,color='blue')
    # for pp in [0.9]:
    pp = sensitivity
    OHF.plot_percentile(all_sims_that_ran,best_cols,[axi[0],axi[1],axi[2]],color='blue',percentile=pp,optimize_output=new_design_parameter)


    axi[0].set_ylim(0,axi[0].get_ylim()[1]*1.1)
    axi[0].set_ylabel(r'Optimal $\epsilon$')

    axi[1].axhline(y=0.05,linestyle='--',color='black',lw=2,marker='')
    axi[1].set_yscale('linear')
    axi[1].set_ylim(0.01,0.09)
    axi[1].set_ylabel(r'Optimal $\widebar{v}_{Cond}$')

    axi[2].set_ylim(0,np.min([1.25*axi[2].get_ylim()[1],500]))
    axi[2].set_ylabel(r'Optimal L ($\mu m$)')


    for axicr in figi.axes:
        axicr.set_xlim(0.1,1.0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='optimization_metric', options=('Cell Volumetric Energy Density (Wh…

In [16]:
class electrode:
    chemistry = 'LVO'
    
    # Initializer / Instance Attributes
    def __init__(self, L, eps, vfc, vfb=0.0):
        self.L = L
        self.eps = eps
        self.vfc = vfc
        self.vfb = vfb
        
    def get_specs(self):
        out_dict = {}
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            out_dict[col] = val
        return out_dict
        
    def nearest_simulated_electrode(self):
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        sims = all_sims_that_ran.copy()
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            col_vals = sims[col].unique()
            sims = sims.loc[sims[col]==col_vals.flat[np.abs(col_vals - val).argmin()]]
        return sims
        
    def performance(self,c_rate):
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        sims = all_sims_that_ran.copy()
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            col_vals = sims[col].unique()
            sims = sims.loc[sims[col]==col_vals.flat[np.abs(col_vals - val).argmin()]]
        return sims.loc[sims['C-rate (1/h)']==c_rate]
        
    def rate_capability(self,ax=None,metric='Capacity (mAh/g)'):
        best_cols = ['Porosity','Vol Frac Cond','L (um)']
        sims = all_sims_that_ran.copy()
        for col,val in zip(best_cols,[self.eps,self.vfc,self.L]):
            col_vals = sims[col].unique()
            sims = sims.loc[sims[col]==col_vals.flat[np.abs(col_vals - val).argmin()]]
        if ax is not None:
            lab = 'L = '+str(int(sims['L (um)'].values[0]))+r', $\epsilon$ =' + str(np.round(sims['Porosity'].values[0],2))+r', $\widebar{v}_{CNT}$ ='+ str(np.round(sims['Vol Frac Cond'].values[0],2))
            ax.plot(sims['C-rate (1/h)'],sims[metric],'-o',label=lab)
            ax.set_xlabel('C-rate (1/h)')
            ax.set_ylabel(metric)
        return sims[['C-rate (1/h)',metric]]
#     def active_material_mass(self):
    
#     def conductor_mass(self):
        
#     def binder_mass(self):
        
#     def active_material_mass_loading(self):
        
#     def volume_distribution(self):
        
#     def tortuosity(self):
        
#     def electronic_conductivity(self):
        
# class cell(electrode):
    

# Rate Capability

This module allows the user to virtually "make" different electrodes and visualize how the discharge capacity will depend on the design parameters chosen. 

The length of the electrode in micros **L_um**, the electrode **porosity**, and the volume fraction of conductive additive **vol_frac_cond** must be specified. 

After the desired values of design parameters are chosen, click the **Run Interact** button to update the plot. 

The plot can be cleared by checking the **clear_plot** box.

In [27]:
choose_eps=widgets.FloatSlider(value=0.5,min=0.1,max=0.9,step=0.01,
    disabled=False,continuous_update=False,
    orientation='horizontal',readout=True,readout_format='.2f')
choose_L=widgets.FloatSlider(value=100,min=50,max=500,step=10,
    disabled=False,continuous_update=False,
    orientation='horizontal',readout=True,readout_format='.0f')
choose_vfc=widgets.FloatSlider(value=0.07,min=0.01,max=0.2,step=0.01,
    disabled=False,continuous_update=False,
    orientation='horizontal',readout=True,readout_format='.2f')

choose_metric = widgets.Dropdown(options = [cap_col,ced_col,aed_col,ed_col,cpd_col,apd_col,pd_col])
                              
button = widgets.Button(description='Plot')
    
clear_button = widgets.ToggleButton(
    value=False,
    description='Clear',
    icon='check'
)

fig2, ax2 = plt.subplots(1, 1, figsize=(10, 4),constrained_layout=True)
fig2.suptitle('Rate Capability')
def plot_RC2(b=None):
    
#     fig2.suptitle('Rate Capability')
    e1 = electrode(L=choose_L.value,eps=choose_eps.value,vfc=choose_vfc.value)
    
    RC_df = e1.rate_capability(ax2,metric=choose_metric.value)
    ax2.grid(True)
    
    colors = plt.cm.plasma(np.linspace(0.3,1,len(ax2.lines)))
    for l,c in zip(ax2.lines,colors):
        l.set_color(c)
        
    ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5),frameon=False)
    
def clear_plot(b=None):
    if clear_button.value:
        ax2.clear()
    
    
@button.on_click
def plot_on_click(b):
    out.clear_output(wait=True)
    with out:
        clear_plot()
        plot_RC2()
        plt.show()
        

out = widgets.Output()
widgets.VBox([widgets.VBox([widgets.Label('Performance Metric:'), choose_metric]),widgets.HBox([widgets.VBox([widgets.Label('Porosity:'), choose_eps]),widgets.VBox([widgets.Label('Electrode Length (um):'), choose_L]),widgets.VBox([widgets.Label('Volume Fraction of Conductor:'), choose_vfc])]),
              widgets.HBox([clear_button,button]),out])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# More Detailed Design Guides

To get a better, more quantitative understanding of the sensitivity of performance to the three design parameters, you can pick a desired performance metric, hold one of the three design parameters fixed, and see a contour map for how the performance depends on the remaining two design parameters.

In [10]:
label_dict = {'Porosity':r'$\epsilon$','L (um)':r'$L$','Vol Frac Cond':r'$\widebar{v}_{cond}$'}

# simple interpolation to fill in NaNs in physically reasonable ranges where there is a lot of data available
def fill_gaps(Z):
    for j,z in enumerate(Z):
        not_nans_indeces = np.where(~np.isnan(z))[0]
        start,end = not_nans_indeces[0],not_nans_indeces[-1]
        z_df = pd.DataFrame(z[start:end+1])
        z_cut = z_df.loc[z_df[0]>=0]
        if len(z_cut)<=2:
            continue
        sim_func = interp1d(z_cut.index.values,z_cut[0].values)
        for i in z_df[z_df.isnull().any(axis=1)].index.values:
            z[start+i] = sim_func(i)
        Z[j] = z
        
    Z = Z.T
    
    for j,z in enumerate(Z):
        not_nans_indeces = np.where(~np.isnan(z))[0]
        start,end = not_nans_indeces[0],not_nans_indeces[-1]
        z_df = pd.DataFrame(z[start:end+1])
        z_cut = z_df.loc[z_df[0]>=0]
        if len(z_cut)<=2:
            continue
        sim_func = interp1d(z_cut.index.values,z_cut[0].values)
        for i in z_df[z_df.isnull().any(axis=1)].index.values:
            z[start+i] = sim_func(i)
        Z[j] = z
        
    return Z.T

In [11]:
fig3,ax3 = plt.subplots(1,1,figsize=(6,5.5),constrained_layout=True)

choose_opt = widgets.Dropdown(options = [ced_col,aed_col,ed_col,cpd_col,apd_col,pd_col,cap_col],
                              description='Performance Metric',style = {'description_width': 'initial'})
choose_crate = widgets.Dropdown(options = all_sims_that_ran['C-rate (1/h)'].unique(),
                                description='C-rate',style = {'description_width': 'initial'})
choose_fixed_col = widgets.Dropdown(options = ['Vol Frac Cond','Porosity','L (um)'],
                                    description='Fixed Design Parameter',style = {'description_width': 'initial'})
choose_fixed_col_val = widgets.Dropdown()

# Define a function that updates the content of y based on what we select for x
def update(*args):
    choose_fixed_col_val.options = all_sims_that_ran[choose_fixed_col.value].unique().tolist()
    choose_fixed_col_val.description = choose_fixed_col.value
    choose_fixed_col_val.style = {'description_width': 'initial'}
choose_fixed_col.observe(update)

# Some function you want executed
def filter_sims(output,c_rate,fixed_col,fixed_col_val):
    
    ax3.clear()
    
    sims_cut = all_sims_that_ran.loc[all_sims_that_ran['C-rate (1/h)']==c_rate]
    sims_cut = sims_cut.loc[sims_cut[fixed_col]==fixed_col_val]

    cols = ['Porosity','L (um)','Vol Frac Cond']
    cols.remove(fixed_col)

    pivtab = sims_cut.pivot_table(index=cols[0], columns=cols[1], values=output)
    
#     dropcols,droprows = [],[]
#     for col in pivtab.columns:
#         if len(pivtab[[col]].dropna()<3): dropcols.append(col)
#     for row in pivtab.index.values:
#         if len(pivtab.loc[row,:].dropna()<3): droprows.append(rows)
#     # still need to do the actual dropping
    
    
    Z = pivtab.T.values
    Z = fill_gaps(Z)
    pivtab = pd.DataFrame(Z.T,columns=pivtab.columns,index=pivtab.index.values)
    
    if output == cap_col:
        output_label = output
    elif int(output.split(')')[0][-1])==3:
        Z*=1000
        output_label = output.split('/')[0] + '/L)'
    elif int(output.split(')')[0][-1])==2:
        Z*=10000
        output_label = output.split('/')[0] + '/m2)'
#     output_label = str(c_rate)+' C ,'+output_label

    X_unique = np.sort(sims_cut[cols[0]].unique())
    Y_unique = np.sort(sims_cut[cols[1]].unique())
    X, Y = np.meshgrid(X_unique, Y_unique)
        
    if (np.max([len(pivtab.dropna(axis=1)),len(pivtab.dropna(axis=0))])<4) or (np.min([len(pivtab),len(pivtab.columns)])<2):
#         display(pivtab)
        print('Not enough data to construct contour map. Please Choose a different combination of inputs')
    else:
        # plot
        xlab,ylab = cols[0],cols[1]
        levels = np.round(np.nanmax(Z)*np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.75,0.8,0.85,0.9,1]),0).astype(int)

        cpf = ax3.contourf(X,Y,Z, levels, cmap=cm.GnBu)

        # Set all level lines to black
        line_colors = ['black' for l in cpf.levels]

        # Make plot and customize axes
        cp = ax3.contour(X, Y, Z, levels=levels, colors=line_colors)
        ax3.clabel(cp, fontsize=10, colors=line_colors,fmt='%1.0f')
        fixed_lab = str(c_rate)+'C, '+label_dict[fixed_col]+' = '+str(fixed_col_val)
        if fixed_col=='L (um)': fixed_lab= fixed_lab + r'$\mu m$'
        ax3.set_title(fixed_lab)
        ax3.set_xlabel(xlab)
        ax3.set_ylabel(ylab)

    #     fig3.colorbar(cpf, ax=ax3)
    
        fig3.suptitle(output_label,fontsize=15)
        
        display(pivtab)
    

widgets.interact(filter_sims,output = choose_opt,
                     c_rate = choose_crate,
                     fixed_col = choose_fixed_col,
                     fixed_col_val = choose_fixed_col_val);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(Dropdown(description='Performance Metric', options=('Cell Volumetric Energy Density (Wh/…